In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score


from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel(
    r"C:\Users\Anu\Desktop\RESEARCH\data2\clinical_biomarker_NF.xlsx"
)

print(df.shape)
df.head()


(1000, 34)


,age,gender,height,weight,occupation,physical_activity_level,living_environment,do_you_currently_experience_knee_pain,pain_score,stiffness,...,does_the_patient_have_any_other_health_conditions_or_risk_factors_that_may_contribute_to_knee_osteoarthritis,what_are_the_suggested_or_ongoing_treatments_for_the_patients_current_condition,koa,does_the_patient_has_obesity,does_the_patient_has_diabetes,does_the_patient_has_hypertension,does_the_patient_has_vitamin_d_deficiency,does_the_patient_has_rheumatoid_arthritis,bmi,koa_grade
0,70.0,Female,1.55,60.0,No,Moderate,Urban,Yes,3.0,Frequently,...,"Obesity or overweight, Family history of osteo...","0 = Lifestyle modification, 1 = Physiotherapy,...",Yes,Yes,No,No,No,No,24.973985,1
1,48.0,Female,1.57,70.0,No,Moderate,Rural,Yes,4.0,Frequently,...,Family history of osteoarthritis,"0 = Lifestyle modification, 1 = Physiotherapy,...",Yes,No,No,No,No,No,28.398718,2
2,54.0,Female,1.58,76.0,No,Moderate,Urban,Yes,3.0,Never,...,"Vitamin D deficiency, Sedentary or low-activit...","0 = Lifestyle modification, 1 = Physiotherapy,...",Yes,No,No,No,Yes,No,30.443839,3
3,52.0,Female,1.55,60.0,No,Low,Urban,Yes,2.0,Occasionally,...,None of the above,"2 = Medication,",Yes,No,No,No,No,No,24.973985,1
4,74.0,Female,1.55,63.0,No,Moderate,Urban,Yes,2.0,Occasionally,...,Vitamin D deficiency,2 = Medication,Yes,No,No,No,Yes,No,26.222685,1


In [3]:
TARGET = "koa_grade"

X = df.drop(columns=[TARGET, "koa_severity"])
y = df[TARGET]

print(X.shape, y.shape)


(1000, 32) (1000,)


In [4]:
num_cols = X.select_dtypes(include=["int64","float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Numeric columns:", num_cols)
print("Categorical columns:", cat_cols)


Numeric columns: ['age', 'height', 'weight', 'pain_score', 'fbs', 'wbc', 'platelets', 'cs', 'cholesterol', 'crp', 'esr', 'rf', 'fbc', 'bmi']
Categorical columns: ['gender', 'occupation', 'physical_activity_level', 'living_environment', 'do_you_currently_experience_knee_pain', 'stiffness', 'have_you_had_any_previous_knee_injuries_(acl_tear,_meniscus_tear,_fracture,_etc.)', 'do_you_experience_swelling_in_your_knees', 'do_you_find_difficulty_in_performing_these_activities_(check_all_that_apply)', 'do_you_have_a_family_history_of_osteoarthritis', 'does_the_patient_have_any_other_health_conditions_or_risk_factors_that_may_contribute_to_knee_osteoarthritis', 'what_are_the_suggested_or_ongoing_treatments_for_the_patients_current_condition', 'koa', 'does_the_patient_has_obesity', 'does_the_patient_has_diabetes', 'does_the_patient_has_hypertension', 'does_the_patient_has_vitamin_d_deficiency', 'does_the_patient_has_rheumatoid_arthritis']


In [5]:
# Ordinal Encoding of stiffness

ordinal_maps = {
    "stiffness": ["never", "occasionally", "frequently", "always"]
}

for col, order in ordinal_maps.items():
    if col in X.columns:
        X[col] = pd.Categorical(X[col], categories=order, ordered=True).codes


In [6]:
# One-Hot Encoding for Nominal Categorical Variables
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

In [7]:
# Feature Scaling (Standardization)
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

In [8]:
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X,
    y,
    test_size=0.20,
    stratify=y,
    random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval,
    y_trainval,
    test_size=0.20,
    stratify=y_trainval,
    random_state=42
)


In [9]:
# Show columns that have NaNs
nan_cols = X_train.columns[X_train.isna().any()].tolist()
print("Columns with NaNs:", nan_cols)



Columns with NaNs: []


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define Logistic Regression model

logreg = LogisticRegression(
    multi_class="multinomial",
    solver="lbfgs",
    class_weight="balanced",
    max_iter=2000,
    random_state=42
)


In [11]:
# Train the model

logreg.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=2000,
                   multi_class='multinomial', random_state=42)

In [12]:
# 3) Evaluate performance

train_pred = logreg.predict(X_train)
val_pred   = logreg.predict(X_val)
test_pred  = logreg.predict(X_test)

print("Logistic Regression Accuracy")
print("Train:", accuracy_score(y_train, train_pred))
print("Val  :", accuracy_score(y_val, val_pred))
print("Test :", accuracy_score(y_test, test_pred))

print("\nClassification Report (Test)")
print(classification_report(y_test, test_pred))

print("\nConfusion Matrix (Test)")
print(confusion_matrix(y_test, test_pred))

Logistic Regression Accuracy
Train: 0.9734375
Val  : 0.91875
Test : 0.93

Classification Report (Test)
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        36
           1       0.91      0.97      0.94        30
           2       0.88      0.88      0.88        52
           3       0.95      0.91      0.93        65
           4       0.94      0.94      0.94        17

    accuracy                           0.93       200
   macro avg       0.93      0.94      0.94       200
weighted avg       0.93      0.93      0.93       200


Confusion Matrix (Test)
[[36  0  0  0  0]
 [ 0 29  1  0  0]
 [ 1  2 46  3  0]
 [ 0  1  4 59  1]
 [ 0  0  1  0 16]]
